# Data Engineering & <br> Datetime Feature Engineering <br> of the Google Store Analytics Dataset
This dataset is provided by the Kaggle competition.  
https://www.kaggle.com/c/ga-customer-revenue-prediction

A lot of columns have data in json format, which need to be split out into separate columns of data.

Also, want to add some further features based on the date/time of day.

In [207]:
import pandas as pd
import numpy as np

from datetime import datetime, timezone, timedelta
# import holidays
import pytz

import requests
import os
gmaps_api_key = os.environ.get('googlemaps_api_key')
timezonedb_api_key = os.environ.get('timezonedb_api_key')

from time import sleep

# Import the Dataset
### (start with just a sample when getting code working)
Instead of working on the full training dataset, which has 903,653 rows, start by working on the top 10,000 rows of the full training dataset. 

In [21]:
# df = pd.read_csv('data/train.csv', low_memory=False, dtype={'fullVisitorId':str}) #full training dataset

df = pd.read_pickle('data/train_sample.pkl') #top 10,000 rows of the training dataset
print(df.shape)
print(df.columns)
df.head(3)

(10000, 12)
Index(['channelGrouping', 'date', 'device', 'fullVisitorId', 'geoNetwork',
       'sessionId', 'socialEngagementType', 'totals', 'trafficSource',
       'visitId', 'visitNumber', 'visitStartTime'],
      dtype='object')


,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western...",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472830385,1,1472830385
1,Organic Search,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472880147,1,1472880147
2,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3895546263509774583,"{""continent"": ""Europe"", ""subContinent"": ""South...",3895546263509774583_1472865386,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472865386,1,1472865386


# Data Engineering - Split out All Columns that Have JSON/Dictionary Info into Individual Columns
Some of the colums in the dataset have json/dictionary info - in order to be able to graph data/break out the variables, we want to separate each of this info to separate columns where each key of the dictionary is its own column data.

The columns that need to be split out are:
['device', 'geoNetwork', 'totals', 'trafficSource']

In [22]:
#print out shape of dataframe and names of all columns of the dataframe to see what columns are getting added as iterate through
print('original df shape: ', df.shape)
print('original df columns: ', df.columns)

#iterate through each of the column names that have to be split out into indiviual columns
for column in ['device', 'geoNetwork', 'totals', 'trafficSource']:

    #print out the name of the column you are processing in this for loop
    print('\n----------------------------------------\nsplitting out column: ', column)

    
    #some of the dictionaries have values of false/true instead of 'false'/'true'; python can't read the false/true
    #because is expecting False/True - instead of creating these boolean names, just convert them to text string
    #of 'false'/'true' before evaluating to dictionary
    df[column] = df[column].map(lambda x: x.replace("false", "'false'"))
    df[column] = df[column].map(lambda x: x.replace("true", "'true'"))
  

    #the JSON/dictionary columns are imported as string types - need to convert to python dictionaries using eval
    df[column] = df[column].map(lambda x: eval(x))

    
    #figure out all the possible unique keys for the dictionaries in this column - need this to make sure we
    #iterate through all of the dictionary data and store into different columns
    unique_column_keys = set()
    #iterate through each row of the column dictionaries
    for row_dictionary in list(df[column]):
        #iterate through each key of the row's dictionary and add to the unique column keys set (use add instead of append for sets)
        for row_key in list(row_dictionary.keys()):
            unique_column_keys.add(row_key)
    print('max length of column dictionaries is: ', len(unique_column_keys))
    print('unique column keys are: ', unique_column_keys)

    
    #use the set of unique_column_keys to add a new column for each of these keys
    for column_key in unique_column_keys:
        #create new column names of format:  originalcolumnname_columnkey, eg 'device_isMobile'
        df[f"{column}_{column_key}"] = df[column].map(lambda x: x.get(column_key, np.nan))
    
    print('updated df shape: ', df.shape)
    print('updated df columns: ', df.columns, '\n')

#drop original columns that had dictionary/jsons so don't keep repetitive data (cuts file size down by almost 40%)
print('\n----------------------------------------\n')
df.drop(['device', 'geoNetwork', 'totals', 'trafficSource'], axis=1, inplace=True)
print('final df shape after dropping original columns: ', df.shape)
    
#show the final df head
df.head(3)

original df shape:  (10000, 12)
original df columns:  Index(['channelGrouping', 'date', 'device', 'fullVisitorId', 'geoNetwork',
       'sessionId', 'socialEngagementType', 'totals', 'trafficSource',
       'visitId', 'visitNumber', 'visitStartTime'],
      dtype='object')

----------------------------------------
splitting out column:  device
max length of column dictionaries is:  16
unique column keys are:  {'operatingSystemVersion', 'operatingSystem', 'deviceCategory', 'browser', 'mobileDeviceBranding', 'flashVersion', 'isMobile', 'mobileDeviceMarketingName', 'screenColors', 'mobileDeviceInfo', 'screenResolution', 'mobileDeviceModel', 'language', 'mobileInputSelector', 'browserVersion', 'browserSize'}
updated df shape:  (10000, 28)
updated df columns:  Index(['channelGrouping', 'date', 'device', 'fullVisitorId', 'geoNetwork',
       'sessionId', 'socialEngagementType', 'totals', 'trafficSource',
       'visitId', 'visitNumber', 'visitStartTime',
       'device_operatingSystemVersion

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device_operatingSystemVersion,device_operatingSystem,...,totals_pageviews,totals_transactionRevenue,trafficSource_campaign,trafficSource_medium,trafficSource_referralPath,trafficSource_isTrueDirect,trafficSource_adContent,trafficSource_source,trafficSource_keyword,trafficSource_adwordsClickInfo
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,not available in demo dataset,Windows,...,1,NaN,(not set),organic,NaN,NaN,NaN,google,(not provided),{'criteriaParameters': 'not available in demo ...
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,not available in demo dataset,Macintosh,...,1,NaN,(not set),organic,NaN,NaN,NaN,google,(not provided),{'criteriaParameters': 'not available in demo ...
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,not available in demo dataset,Windows,...,1,NaN,(not set),organic,NaN,NaN,NaN,google,(not provided),{'criteriaParameters': 'not available in demo ...


# Datetime Feature Engineering
### (Add in Additional Date/Time Features into our Data to Help with Analysis - day of week, holiday, local timezone, etc.)
1. Use the visitStartTime, which is a POSIX timestamp to get the UTC date and hour all together into a datetime object (rather than converting just the date column, which doesn't have any hour info).  This POSIX format can be extracted to a python datetime object - for example, a timestamp of 1472830385 becomes datetime.datetime(2016, 09, 02, 9, 33, 5), wich is in format year, month, day, hour, minute, second.
2. Then use the UTC datetime object to extract to a string formatted datetime_iso_utc (that can be used for time series analysis based on purchase time).  (iso format is: 'YYYY-MM-DD HH:MM:SS')
    - datetime_iso_utc
3. Convert to local timezone datetime object (this will require an indepth sub-process to identify timezone/call it)
4. Using the local timezone datetime object, extract multiple new datetime features:
    - datetime_iso_local
    - year_local
    - month_local
    - day_local
    - yearday_local (overall day of the year 1 to 365, which captures the month and day values)
    - weekday_local (values of 1 to 7 - 1 is Monday)
    - hour_local
5. Try to add a column to determine whether the date is a holiday or not.
    - holiday_local


### Figure Out Local Timezones for Each Location 
- Combine City/Country info given to us
- Get a rough lat/lng for that location by using Gmaps Geocoding API
- Use timezonedb API to send each location lat/lng to find out the local timezone

In [152]:
###CREATE NEW CITY_COUNTRY COLUMN THAT COMBINES CITY/COUNTRY TOGETHER###

#the geographic information provided almost always has a country listed (a very small percent country is "(not set)")
#note - there some strange, incorrect combinations - like New York Canada or Mountain View Germany, which will have a small percentage of rough data
#the geographic information sometimes have a city (more common for larger countries, but not consistent)

df['city_country'] = df.apply(lambda row: 
                              row.geoNetwork_city + " " + row.geoNetwork_country 
                              if row.geoNetwork_city not in ["not available in demo dataset", "(not set)"] 
                              else row.geoNetwork_country,
                              axis='columns')

In [102]:
###GET LAT, LNG OF ALL CITY_COUNTRY COMBINATIONS (best approximation for location with sample data)###

#create a unique set of all city_country combinations to cycle through
city_country_set = set(df.city_country)
#remove the '(not set)' label from the set for the few cases where '(not set)' was in country
city_country_set.remove('(not set)')

#get Lat/Lng for our locations by using Google Geocoding and store info in a dictionary
city_country_lat_lng = {}

for city_country in city_country_set:
    #Google Geocoding works by inputting an address and get a lat, lng returned
    #https://maps.googleapis.com/maps/api/geocode/json?address=1600+Amphitheatre+Parkway,+Mountain+View,+CA&key=YOUR_API_KEY
    #every data point at least has a country

    base_geocode_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
    geocode_parameters = {'address': city_country,
                          'key': gmaps_api_key}

    json_geocode_response = requests.get(base_geocode_url, geocode_parameters).json()
    lat = json_geocode_response.get('results')[0].get('geometry').get('location').get('lat')
    lng = json_geocode_response.get('results')[0].get('geometry').get('location').get('lng')
    
    city_country_lat_lng[city_country] = (lat, lng)


#add a column to the dataframe that has each city_country approximate lat/lng
df['lat_lng'] = df['city_country'].map(lambda x: city_country_lat_lng.get(x, np.nan))

In [188]:
###GET APPROXIMATE TIMEZONE FOR EACH CITY_COUNTRY COMBINATIONS (using timezonedb API)###

#store timezone for all of our city_country locations in a dictionary
city_country_timezone = {}

#city_country_lat_lng is a dictionary with city_country: (lat, lng)
#cycle through each city_country lat,lng and call the timezonedb API
for city_country, lat_lng in city_country_lat_lng.items():
    #get local timezone info for each city/country combination in our dataset by looking up their lat,lng
    #http://api.timezonedb.com/v2.1/get-time-zone (has a 1 second rate limit)
    base_timezonedb_url = 'http://api.timezonedb.com/v2.1/get-time-zone?'
    timezonedb_parameters = {'key': timezonedb_api_key,
                             'format': 'json',
                             'by': 'position',
                             'lat': lat_lng[0],
                             'lng': lat_lng[1]}

    json_timezone_response = requests.get(base_timezonedb_url, timezonedb_parameters).json()
    #get the gmt timezone offset and the abbreviation for the timezone - gmtOffset it the time offset in seconds
    #so will need to divide it by 3600 to get the usable offset in hours
    timezone_abbreviation = json_timezone_response.get('abbreviation', np.nan)
    timezone_gmt_offset = json_timezone_response.get('gmtOffset', np.nan)/3600
    
    city_country_timezone[city_country] = (timezone_abbreviation, timezone_gmt_offset)
    
    #need to sleep for 1 second because api says it has a 1 request/second rate limit
    sleep(1.1)

#add a column to the dataframe that has each city_country approximate timezone
df['timezone'] = df['city_country'].map(lambda x: city_country_timezone.get(x, (np.nan, np.nan)))

In [402]:
df[df.fullVisitorId == '7020148216329907379']

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device_operatingSystemVersion,device_operatingSystem,...,trafficSource_medium,trafficSource_referralPath,trafficSource_isTrueDirect,trafficSource_adContent,trafficSource_source,trafficSource_keyword,trafficSource_adwordsClickInfo,city_country,lat_lng,timezone
309,Referral,20160902,7020148216329907379,7020148216329907379_1472854128,Not Socially Engaged,1472854128,1,1472854128,not available in demo dataset,Chrome OS,...,referral,/,NaN,NaN,mall.googleplex.com,NaN,{'criteriaParameters': 'not available in demo ...,Mountain View United States,"(37.3860517, -122.0838511)","(PDT, -7.0)"


In [403]:
# df.timezone[310]
df.iloc[310]

channelGrouping                                                              Referral
date                                                                         20160902
fullVisitorId                                                      149633083723377298
sessionId                                               149633083723377298_1472819179
socialEngagementType                                             Not Socially Engaged
visitId                                                                    1472819179
visitNumber                                                                         1
visitStartTime                                                             1472819179
device_operatingSystemVersion                           not available in demo dataset
device_operatingSystem                                                        Windows
device_deviceCategory                                                         desktop
device_browser                                        

In [368]:
city_country_timezone

{'Greenland': ('', 0.0),
 'Cyprus': ('EEST', 3.0),
 'Prague Czechia': ('CEST', 2.0),
 'Mountain View China': ('PDT', -7.0),
 'Botswana': ('CAT', 2.0),
 'London United States': ('EDT', -4.0),
 'Poland': ('CEST', 2.0),
 'Belgium': ('CEST', 2.0),
 'Warsaw Poland': ('CEST', 2.0),
 'Lebanon': ('EEST', 3.0),
 'Karachi Pakistan': ('PKT', 5.0),
 'Vietnam': ('+07', 7.0),
 'Lithuania': ('EEST', 3.0),
 'Nepal': ('+0545', 5.75),
 'Croatia': ('CEST', 2.0),
 'Panama': ('EST', -5.0),
 'Iraq': ('+03', 3.0),
 'Kazakhstan': ('+06', 6.0),
 'Mexico': ('CDT', -5.0),
 'Malaysia': ('+08', 8.0),
 'Columbus United States': ('EDT', -4.0),
 'Hong Kong': ('HKT', 8.0),
 'Kuwait': ('+03', 3.0),
 'San Jose United States': ('PDT', -7.0),
 'Chennai India': ('IST', 5.5),
 'Madrid Spain': ('CEST', 2.0),
 'Ukraine': ('EEST', 3.0),
 'Bulgaria': ('EEST', 3.0),
 'Laos': ('+07', 7.0),
 'Kiev Ukraine': ('EEST', 3.0),
 'Senegal': ('GMT', 0.0),
 'Tunis Tunisia': ('CET', 1.0),
 'Costa Rica': ('CST', -6.0),
 'Chuo Japan': ('JST',

### Extract the Different Datetime Features

- datetime_iso_utc   ('YYYY-MM-DD HH:MM:SS')
- datetime_iso_local  ('YYYY-MM-DD HH:MM:SS')
- year_local
- month_local
- day_local
- yearday_local (overall day of the year 1 to 365, which captures the month and day values)
- weekday_local  (values of 1 to 7 where 1 is Monday)
- hour_local
- holiday_local

In [431]:
def extract_datetime_info(timestamp, gtm_offset_hours):
    #first convert the timestamp to datetime object with utc setting
    datetime_object_utc = datetime.fromtimestamp(timestamp, tz=pytz.UTC)
    #convert timestamp to isoformat for timeseries analysis in Tableau, etc. (and ease of viewing) - both utc and local
    #initially gives string format of '2016-09-02T09:33:05', strip out the middle T so looks like '2016-09-02 09:33:05'
    datetime_iso_utc = datetime_object_utc.isoformat().replace("T", " ")  #'YYYY-MM-DD HH:MM:SS'
    
    #these timestamps aren't adjusted for timezone, in order to have better analysis
    #it seems like we would want the datetime in the approximate local timezone based on country/city since
    #we would assume consumer behavior is more similar between consumers based on the local time, rather than UTC time
    #some of the data doesn't have a timezone, so only return the datetime_iso_utc if gtm_offset_hours is not a nan
    if ~np.isnan(gtm_offset_hours): 
        #convert datetime by changing the timezone by the gtmOffsets
        datetime_object_local = datetime_object_utc.astimezone(timezone(timedelta(hours=gtm_offset_hours)))

        #convert timestamp to isoformat for timeseries analysis in Tableau, etc. (and ease of viewing) - both utc and local
        #initially gives string format of '2016-09-02T09:33:05', strip out the middle T so looks like '2016-09-02 09:33:05'
        datetime_iso_local = datetime_object_local.isoformat().replace("T", " ")  #'YYYY-MM-DD HH:MM:SS'

        #extract year, month, month-day, day, day of the week (using isoweekday), and hour into different variables
        year_local = datetime_object_local.year #YYYY (as int)
        month_local = datetime_object_local.month #MM (as int)
        day_local = datetime_object_local.day #DD (as int) - this is the day of the month, max 31
        yearday_local = datetime_object_local.timetuple().tm_yday #DDD (as int) - this is the day of the year sequentially 1 to 365 which gives context of the month-day in one value
        weekday_local = datetime_object_local.isoweekday() #integer values of 1 to 7 where 1 is Monday
        hour_local = datetime_object_local.hour #HH (as int)

        return datetime_iso_utc, datetime_iso_local, year_local, month_local, day_local, yearday_local, weekday_local, hour_local
    
    else:
        return datetime_iso_utc, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan

In [432]:
#use the extract_datetime_info function to add columns of all of the additional datetime features to the df
df[['datetime_iso_utc', 'datetime_iso_local',
    'year_local', 'month_local','day_local',
    'yearday_local', 'weekday_local', 'hour_local']] = df[['visitStartTime', 'timezone']].apply(lambda row:
                                                        extract_datetime_info(row.visitStartTime, row.timezone[1]),
                                                        axis='columns',
                                                        result_type='expand')

In [439]:
df.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device_operatingSystemVersion,device_operatingSystem,...,lat_lng,timezone,datetime_iso_utc,datetime_iso_local,year_local,month_local,day_local,yearday_local,weekday_local,hour_local
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,not available in demo dataset,Windows,...,"(38.423734, 27.142826)","(+03, 3.0)",2016-09-02 15:33:05+00:00,2016-09-02 18:33:05+03:00,2016.0,9.0,2.0,246.0,5.0,18.0
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,not available in demo dataset,Macintosh,...,"(-25.274398, 133.775136)","(ACST, 9.5)",2016-09-03 05:22:27+00:00,2016-09-03 14:52:27+09:30,2016.0,9.0,3.0,247.0,6.0,14.0
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,not available in demo dataset,Windows,...,"(40.4167754, -3.7037902)","(CEST, 2.0)",2016-09-03 01:16:26+00:00,2016-09-03 03:16:26+02:00,2016.0,9.0,3.0,247.0,6.0,3.0
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,not available in demo dataset,Linux,...,"(-0.789275, 113.921327)","(WIB, 7.0)",2016-09-03 05:40:13+00:00,2016-09-03 12:40:13+07:00,2016.0,9.0,3.0,247.0,6.0,12.0
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,not available in demo dataset,Android,...,"(55.378051, -3.435973)","(BST, 1.0)",2016-09-02 13:23:20+00:00,2016-09-02 14:23:20+01:00,2016.0,9.0,2.0,246.0,5.0,14.0


# Clean and Save Engineered Dataframe to Pickle for Easier Access Later

In [449]:
#there are many columns in our sample data that only have 'not available in demo dataset' values,
#so want to drop those columns as they are not useful
columns_to_drop = ['device_language', 'device_screenColors', 'device_mobileDeviceMarketingName', 'device_mobileInputSelector', 
'device_mobileDeviceBranding', 'device_mobileDeviceInfo', 'device_browserSize', 'device_browserVersion', 'device_mobileDeviceModel',
'device_flashVersion', 'device_screenResolution', 'device_operatingSystemVersion', 'geoNetwork_cityId', 'geoNetwork_latitude',
'geoNetwork_networkLocation', 'geoNetwork_longitude', 'trafficSource_adwordsClickInfo']

df.drop(columns_to_drop, axis=1, inplace=True)

In [450]:
# #dropping one row in the FULL TRAINING DATASET (v1) that was causing an error when trying to save data to a csv for Tableau processing
# df.drop(564203, inplace=True)

In [452]:
#verify the size of the df we are saving (the full training datset should be about 100k rows on partial
#and 900k rows on fulland 43 good columns)
print(df.shape)

(10000, 43)


In [453]:
#save the dataframe to pickle and a csv - PARTIAL DATASET
df.to_pickle('data/train_partial_data_split.pkl')
df.to_csv('data/train_partial_data_split.csv')

# #save the dataframe to pickle and a csv - FULL TRAINING DATASET (v1)
# df.to_pickle('data/train_v1_full_data_split.pkl')
# df.to_csv('data/train_v1_full_data_split.csv')

# Initial Exploration - Determine the Value Counts of each Column
To get a better grasp of the data that we have, look at value counts of each column.  Some will have lots of different values (like fullVisitorId, which will be unique to every individual).  Others will have binary info.

Cycle through and print the top 10 value_counts for each column.

In [25]:
#iterate through every single column in the dataset (that is not a dictionary, can't do value_counts on a dictionary)
for column in df.columns:

    #print out the name of the column you are processing in this for loop
    print('\n----------------------------------------\nCOLUMN: ', column)
    
    #if column has dictionaries, it can't do value_counts so flag this in output
    if type(df[column][0]) == dict:
        print('column contains dictionaries')
        print('example dictionary from 0th row: \n', df[column][0])
    
    else:
        #print out the length of the value_counts for the column
        print('# of Unique Values in Column: ', len(df[column].value_counts()))

        #print out the top ten value_counts for the column
        #normalize=True provides relative frequency of the value (instead of just count), so 0.473 is 47.3%
        print('Top 10 Value Counts for Column: \n', df[column].value_counts(normalize=True).head(10))



----------------------------------------
COLUMN:  channelGrouping
# of Unique Values in Column:  7
Top 10 Value Counts for Column: 
 Organic Search    0.4730
Social            0.1851
Direct            0.1645
Referral          0.1190
Paid Search       0.0314
Affiliates        0.0201
Display           0.0069
Name: channelGrouping, dtype: float64

----------------------------------------
COLUMN:  date
# of Unique Values in Column:  5
Top 10 Value Counts for Column: 
 20160902    0.2613
20170126    0.2209
20170623    0.2095
20170312    0.1781
20170203    0.1302
Name: date, dtype: float64

----------------------------------------
COLUMN:  fullVisitorId
# of Unique Values in Column:  9251
Top 10 Value Counts for Column: 
 384990845055862114     0.0007
9127817804140120297    0.0007
7560731149412661485    0.0006
3525537916960843419    0.0005
1301232867629805291    0.0005
4127918227876727095    0.0005
1957458976293878100    0.0005
6693884468685098722    0.0005
2994779313411060498    0.0005
850

COLUMN:  geoNetwork_metro
# of Unique Values in Column:  22
Top 10 Value Counts for Column: 
 not available in demo dataset        0.5941
(not set)                            0.1865
San Francisco-Oakland-San Jose CA    0.1103
New York NY                          0.0274
Los Angeles CA                       0.0134
London                               0.0122
Chicago IL                           0.0105
Seattle-Tacoma WA                    0.0080
Columbus OH                          0.0069
Atlanta GA                           0.0039
Name: geoNetwork_metro, dtype: float64

----------------------------------------
COLUMN:  geoNetwork_networkLocation
# of Unique Values in Column:  1
Top 10 Value Counts for Column: 
 not available in demo dataset    1.0
Name: geoNetwork_networkLocation, dtype: float64

----------------------------------------
COLUMN:  geoNetwork_continent
# of Unique Values in Column:  6
Top 10 Value Counts for Column: 
 Americas     0.5235
Europe       0.2375
Asia         0.20

#### Look more closely at fullVisitorID
##### (this is the id we need to assign final revenue to)

In [178]:
#for fullVisitorID check out actual value counts (instead of percentage) - this is the id we need to assign final revenue to
print('\n----------------------------------------\nCOLUMN: ', 'fullVisitorId', '\n')

print('# of Unique Visitors: ', len(df['fullVisitorId'].value_counts()), '\n')

print('# of Unique Visitors Tracked More than Once: ', len(df['fullVisitorId'].value_counts()[df['fullVisitorId'].value_counts()>1]), '\n')

print('Top 10 Visitors/Counts: \n', df['fullVisitorId'].value_counts().head(10), '\n')

print('Last 5 Visitors/Counts: \n', df['fullVisitorId'].value_counts().tail(5))


----------------------------------------
COLUMN:  fullVisitorId 

# of Unique Visitors:  9251 

# of Unique Visitors Tracked More than Once:  609 

Top 10 Visitors/Counts: 
 384990845055862114     7
9127817804140120297    7
7560731149412661485    6
3525537916960843419    5
1301232867629805291    5
4127918227876727095    5
1957458976293878100    5
6693884468685098722    5
2994779313411060498    5
8507202977688501909    4
Name: fullVisitorId, dtype: int64 

Last 5 Visitors/Counts: 
 1715754924747762270    1
4879078049194301640    1
7269187277725036941    1
0580786618955798041    1
6722858859530306006    1
Name: fullVisitorId, dtype: int64
